## Note : This notebook was prepared for a previous NLP course by Brian Chapman, Wendy Chapman, Jianlin Shi and Danielle Mowery

# Clinical natural language processing

Clinical texts contain a summary of the patient's clinical story and their physiological status changes over time. Specifically, members of the care providing team document these changes by describing the patient's status upon admission, during examination, throughout their treatment, and upon discharge/transfer. What kinds of secondary use (beyond patient care) could be supported from information extracted from textual data e.g., clinical reports?

* Recruit patients for clinical trials
* Enrich structured problem lists with missed problems
* Build cohorts for comparative effectiveness studies
* Extract a patient's family history information for assessing predisposition of disease


Below is a sample report from <a href=http://www.mtsamples.com/site/pages/sample.asp?Type=91-SOAP%20/%20Chart%20/%20Progress%20Notes&Sample=40-Cardiology%20Progress%20Note>"MTSamples.com"</a>

![MTSamples.com](./images/MT_Samples_report.png)

From the report above, how would you tell me whether the patient has had chest pain? What information did you use to induce that?

# Characterizing textual reports

What information is described in the report above?
* symptoms
* signs
* tests
* treatments

What type of structures are used to construct the report?
* words
* phrases
* sentences
* sections
* SOAP structure

How does this report structure differ from other texts e.g., newspaper articles? How is it similar? 

In this lesson, we are going to learn how to analyze information within reports that can be leveraged to distinguish report types from each other (discharge summary vs radiology reports), classify patients (diseased vs healthy), and other use cases.


# Reading in files

Again, one of the fundamental steps to processing and analyzing textual reports, is locating files to read into memory. To get started we are going to call in or 'import' some tools that allow us to read in files from the computer.

In [1]:
import glob, os
print(os.getcwd()+r"/data/mtsamples_documents")

/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents


# Reading in texts from the directory

Next, we want to read in a set of text files using glob to begin processing them. Glob will return us a list of text files with their corresponding paths.  

In [2]:
files=glob.glob(os.getcwd()+"/data/mtsamples_documents/*.txt")
print(files)

['/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents/289-Dermatitis - SOAP.txt', '/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents/1602-SOAP - Shortness of Breath.txt', '/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents/73-Gen Med SOAP - 3.txt', '/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents/1601-SOAP - Lung Mass.txt', '/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents/1439-SOAP - Substance Abuse.txt', '/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents/450-URI - SOAP.txt', '/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents/452-Fifth Disease - SOAP 2.txt', '/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents/1198-SOAP - Temporal Mass 2.txt', '/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents/288-Acne - SOAP 2.txt', '/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents/37-Gen Med SOAP - 2.txt', '/home/jianlins/work/decart

# Selecting a file for processing

Using the list and indexing, we can select a file containing a progress note report to process. Use the index to select a report of interest to you. We will select the seventh file in the list from the end. 

In [3]:
files[-7]

'/home/jianlins/work/decart_rule_based_nlp/data/mtsamples_documents/1440-Gen Med SOAP.txt'

# Opening a file for processing

Now that we've selected a file to process, we can use 'open' and 'read' to read all the file contents (report) into memory.

In [4]:
report_text=open(files[-7]).read()
print(report_text)

Sample Name: Gen Med SOAP  Description:  Palpitations, possibly related to anxiety. Fatigue. Loose stools with some green color and also some nausea.
(Medical Transcription Sample Report)
SUBJECTIVE:  This patient presents to the office today because he has not been feeling well.  He was in for a complete physical on 05/02/2008.  According to the chart, the patient gives a history of feeling bad for about two weeks.  At first he thought it was stress and anxiety and then he became worried it was something else.  He says he is having a lot of palpitations.  He gets a fluttering feeling in his chest.  He has been very tired over two weeks as well.  His job has been really getting to him.  He has been feeling nervous and anxious.  It seems like when he is feeling stressed he has more palpitations, sometimes they cause chest pain.  These symptoms are not triggered by exertion.  He had similar symptoms about 9 or 10 years ago.  At that time he went through a full workup.  Everything ended u

# Segmenting a report 

One of the fundamental tasks to text processing is segmenting the text into more managable parts e.g., section segmentation or sentence segmentation. For many NLP tasks such as information extraction, a single sentence can contain most information about an entity or event. Using Textblob, we can segment the report into sentences. 

In [5]:
from textblob import TextBlob

#sentence object
blob = TextBlob(report_text)

for sent in blob.sentences:
    print(sent)
    print("-----------------------")

Sample Name: Gen Med SOAP  Description:  Palpitations, possibly related to anxiety.
-----------------------
Fatigue.
-----------------------
Loose stools with some green color and also some nausea.
-----------------------
(Medical Transcription Sample Report)
SUBJECTIVE:  This patient presents to the office today because he has not been feeling well.
-----------------------
He was in for a complete physical on 05/02/2008.
-----------------------
According to the chart, the patient gives a history of feeling bad for about two weeks.
-----------------------
At first he thought it was stress and anxiety and then he became worried it was something else.
-----------------------
He says he is having a lot of palpitations.
-----------------------
He gets a fluttering feeling in his chest.
-----------------------
He has been very tired over two weeks as well.
-----------------------
His job has been really getting to him.
-----------------------
He has been feeling nervous and anxious.
-------

# Characterizing sentences

Because Textblob returns a list, we can also count the number of sentences in the report. How many sentences are there in the report? How might knowing the number of sentences be important to guessing a report type? 

We can also obtain a particular sentence in the list using the index. What does the 10th sentence read? When could it be important to select a sentence by its index?

In [6]:
print(len(blob.sentences))

print(blob.sentences[10])

65
His job has been really getting to him.


# Tokenizing sentences into words

Words are the basic building blocks of text. As humans, when we want to find relevant information from text we search text for words in sentences of the report related to the topic of interest to us. Textblob supports tokenization by providing the words or tokens from the report text. What's the difference between these two functions?


In [7]:
print(blob.words[0:50])

print("=================")

print(blob.tokens[0:50])

['Sample', 'Name', 'Gen', 'Med', 'SOAP', 'Description', 'Palpitations', 'possibly', 'related', 'to', 'anxiety', 'Fatigue', 'Loose', 'stools', 'with', 'some', 'green', 'color', 'and', 'also', 'some', 'nausea', 'Medical', 'Transcription', 'Sample', 'Report', 'SUBJECTIVE', 'This', 'patient', 'presents', 'to', 'the', 'office', 'today', 'because', 'he', 'has', 'not', 'been', 'feeling', 'well', 'He', 'was', 'in', 'for', 'a', 'complete', 'physical', 'on', '05/02/2008']
['Sample', 'Name', ':', 'Gen', 'Med', 'SOAP', 'Description', ':', 'Palpitations', ',', 'possibly', 'related', 'to', 'anxiety', '.', 'Fatigue', '.', 'Loose', 'stools', 'with', 'some', 'green', 'color', 'and', 'also', 'some', 'nausea', '.', '(', 'Medical', 'Transcription', 'Sample', 'Report', ')', 'SUBJECTIVE', ':', 'This', 'patient', 'presents', 'to', 'the', 'office', 'today', 'because', 'he', 'has', 'not', 'been', 'feeling', 'well']


# Counting words in the text

In some cases, knowing the frequency of words in text can help us better understand the content of the text and classify it by topic. For example, if we know a report contains many terms associated with the heart e.g., cardiology, echocardiogram, heart, infarction, we may deduce that the patient may have been seen for cardiovascular problems. If we review patient satisfaction surveys with many positive terms, we may deduce that the patient had a satisfactory visit. Textblob can give us word counts from a report text. Which words are most frequent? Are they always informative? HINT: How frequent is the word 'and'?

In [8]:
print(blob.word_counts)


print("--------------------------------")
print("and:",blob.words.count('and'))

defaultdict(<class 'int'>, {'sample': 2, 'name': 1, 'gen': 1, 'med': 1, 'soap': 1, 'description': 1, 'palpitations': 4, 'possibly': 3, 'related': 3, 'to': 24, 'anxiety': 5, 'fatigue': 2, 'loose': 3, 'stools': 3, 'with': 7, 'some': 10, 'green': 3, 'color': 3, 'and': 23, 'also': 5, 'nausea': 4, 'medical': 2, 'transcription': 1, 'report': 1, 'subjective': 1, 'this': 3, 'patient': 4, 'presents': 1, 'the': 21, 'office': 4, 'today': 4, 'because': 1, 'he': 38, 'has': 12, 'not': 6, 'been': 9, 'feeling': 5, 'well': 2, 'was': 7, 'in': 12, 'for': 11, 'a': 16, 'complete': 1, 'physical': 1, 'on': 5, '05/02/2008': 1, 'according': 1, 'chart': 2, 'gives': 1, 'history': 1, 'of': 9, 'bad': 2, 'about': 6, 'two': 4, 'weeks': 3, 'at': 4, 'first': 1, 'thought': 1, 'it': 6, 'stress': 4, 'then': 1, 'became': 1, 'worried': 1, 'something': 3, 'else': 1, 'says': 3, 'is': 18, 'having': 3, 'lot': 1, 'gets': 1, 'fluttering': 1, 'his': 13, 'chest': 3, 'very': 3, 'tired': 1, 'over': 3, 'as': 3, 'job': 1, 'really': 2,

# Coding exercise I

What are the 5 most frequent words in the dictionary?

In [9]:
# your code here


# Tokenizing multi-words (n-grams)

Signs, symptoms, diagnoses are often made from more than one word. In addition to individual words, we can define windows of words with varying lengths of tokens (1-token = unigrams; 2-token = bigrams, etc. 

In [ ]:
print("Bigrams:", blob.ngrams(n=2))
print("Trigram - right upper extremity:", blob.ngrams(n=3).count(['right', 'upper', 'extremity']))

# Lemmatizing words to their stems (morphology)

Words can have many variants based on parts of speech. Textblob supports lemmatizing words to their root form. This can be important for identifying a concept documented with many lexical variants.

In [ ]:
# before we perform syntactic operations like lemmatization we should make sure that we have some resources available
# including WordNet and some taggers
import nltk
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download("brown")

In [ ]:
from textblob import Word
w = Word("smokes")
print("lemma of smokes is:", w.lemmatize())

How well does a lemmatizer trained on general English work for medical language?

# Tagging parts of speech (syntax)

Signs, symptoms, diagnoses are often nouns. Textblob can give us the parts of speech of words from the text. There are two sets of tags commonly used for part of speech tagging: [Penn Treebank] (https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html") and [Brown Corpus tags] (http://www.comp.leeds.ac.uk/ccalas/tagsets/brown.html) Let's briefly review the different standards for tagging parts of speech of text. Which tag set appears below?

In [ ]:
print(blob.tags)

# Parsing noun phrases

Not only are signs, symptoms, diagnoses often nouns, but they can be mulit-word and part of noun phrases; therefore, parsing phrases based on part of speech can help us identify words that when grouped together describe a salient clinical concept e.g., 'coronary artery disease' --> ('coronary', 'JJ'), ('artery', 'NN'), ('disease', 'NN') rather than just 'coronary' ('coronary', 'JJ'). What types clinical concepts are most frequently represented by noun phrases in the text? Which seem correct and which don't seem correct?

In [ ]:
print(blob.np_counts)

# Coding exercise III

Print the noun phrases that occur more than 1 time in the text above.

In [ ]:
# your code here

# Viewing noun phrases in text

Part of speech tags in conjuction with named entity recognition labels can depict how clinical concepts are created from words create phrases and clauses. Can you identify the beginning (B), inside (I) and outside (O) of each phrase? In addition to BIO labelling there are other labels standards. [Take a look!] (https://lingpipe-blog.com/2009/10/14/coding-chunkers-as-taggers-io-bio-bmewo-and-bmewo/) 

Go here to experiment with parse trees: foxtype.com/sentence-tree

In [ ]:
blob.parse()

# Determining meaning of words (semantics)

Words, phrases, and clauses convey meaning or the semantics of text by describing entities and events within clinical text. The meaning of a concept can be derived from a lexicon or dictionary of words.
For instance, we can define the concept of Addison’s disease with its synonyms (words sharing similar meaning) e.g.,
adrenocortical insufficiency. 

In [ ]:
vertigo = Word('vertigo')
print("Vertigo: ",vertigo.definitions)


dizziness = Word('dizziness')
print("Dizziness: ",vertigo.definitions)

Words have synonyms, and there are many dictionaries that will provide synonyms. WordNet is the most comprehensive but the least focused on medical words (although it has many medical words). The UMLS Metathesaurus is a thesaurus of thesauri that has synonyms for many medical and biological terms.

Check out the synonymize function someone wrote: rwet.decontextualize.com/book/textblob/

In [ ]:
from textblob.wordnet import NOUN
synsets = dizziness.get_synsets(pos=NOUN)
print(synsets[0].lemma_names())

# Identifying many senses of a word

In addition to one word being similar to another word, one word can convey many meanings (i.e., a word can be polysemous), e.g., discharge can be release of a patient from care or a substance from an abscess. Discerning the sense or meaning of a word is called word sense disambiguation.  Using Textblob and the underlying dictionary <a href="https://wordnet.princeton.edu/wordnet/">WordNet</a>), we can obtain the definition and corresponding synsets (i.e., word senses) of a word. Are any of the synsets for 'admit' below clinical? 

In [ ]:
from textblob.wordnet import VERB

word = Word('admit')
for definition, syn in zip(word.definitions, word.get_synsets(pos=VERB)):
    print(definition+" --> "+str(syn)+"\n")

# Determine the degree of similarity between synsets of two words

Given two words, we can also compare the synsets to determine the degree of semantic similarity. How similar are the pus and discharge synsets below? Can you think of other words with more semantic similarity? Try them! Test different v. of the same word to look for similarity.

In [ ]:
from textblob.wordnet import Synset

declare_true=Synset('admit.v.01')
give_access=Synset('admit.v.05')
print("Similarity: ", declare_true.path_similarity(give_access))

# Synonyms
Use the same code to see how similar two different words are

In [ ]:
declare_true=Synset('dizziness.n.01')
give_access=Synset('vertigo.n.01')
print("Similarity: ", declare_true.path_similarity(give_access))